In [1]:
import pandas as pd
import numpy as np
import typing
save_plot = True
debug = True
data_dir = '../data/dianli.xlsx'
data = pd.read_excel(data_dir,nrows=1000 if debug else None)

raw_data_copy = data.copy()
raw_data_copy.columns = ['date', 'hour',
                         'f1', 'f2', 'f3', 'f4', 'f5', 'target']
raw_data_copy = raw_data_copy[['f1', 'f2', 'f3', 'f4', 'f5', 'target']]
targ_cols = ("target",)  # NDX是我们需要预测的值

# 数据预处理
class TrainConfig(typing.NamedTuple):
    T: int
    train_size: int
    batch_size: int
    loss_func: typing.Callable


class TrainData(typing.NamedTuple):
    feats: np.ndarray
    targs: np.ndarray
    
from sklearn.preprocessing import StandardScaler
def read2Dataloader(data,col_names):
    scaler = StandardScaler().fit(data)
    proc_dat = scaler.transform(data)
    mask = np.ones(proc_dat.shape[1],dtype=bool)
    dat_cols = list(data.columns)
    for col_name in col_names:
        mask[dat_cols.index(col_name)] = False
    feats = proc_dat[:,mask]
    targs = proc_dat[:,~mask]
    return TrainData(feats,targs)
# 返回特征
trainData = read2Dataloader(raw_data_copy,targ_cols)

In [10]:
'''
基本网络类型
'''
import torch.nn as nn
import json
from torch import optim
import collections
import typing
import torch.nn.functional as F
'''
初始化简单的lstm网络
'''
RnnNet = collections.namedtuple("RnnNet",["rnn","rnn_optimizer"])
RnnAttNet = collections.namedtuple("RnnAttNet",["rnn","rnn_optimizer"])

class Lstm(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers , output_size , dropout, batch_first):
        super(Lstm, self).__init__()
        # lstm的输入 #batch,seq_len, input_size
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout
        self.batch_first = batch_first

        self.rnn = nn.LSTM(input_size=input_size,hidden_size=hidden_size,
                        num_layers=num_layers,batch_first=True)
        # self.rnn = nn.LSTM(input_size=self.input_size, 
        #                    hidden_size=self.hidden_size, 
        #                    num_layers=self.num_layers, 
        #                    batch_first=self.batch_first, 
        #                    dropout=self.dropout)
        self.linear = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x):
       # print('x:shape',x.shape)
        out, (hidden, cell) = self.rnn(x)  # x.shape : batch, seq_len, hidden_size , hn.shape and cn.shape : num_layes * direction_numbers, batch, hidden_size
        # a, b, c = hidden.shape
        # out = self.linear(hidden.reshape(a * b, c))
       # print('out shape:',out.shape,'hidden shape:',hidden.shape,'cell shape:' ,cell.shape)
        out = self.linear(hidden)
        print(out.shape)

        return out

In [48]:



import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as tf
class BiLSTM_Attention(nn.Module):
    def __init__(self,input_size: int, hidden_size: int, T: int,bidirectional: True,num_classes: int):
        super(BiLSTM_Attention, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.T = T
        self.bidirectional = bidirectional
        self.num_classes = num_classes
        self.lstm = nn.LSTM(input_size=input_size,hidden_size=hidden_size,
                    num_layers=1,batch_first=True,bidirectional=self.bidirectional)
        self.tanh1 = nn.Tanh()
        self.w = nn.Parameter(torch.zeros(hidden_size * 2))
        self.tanh2 = nn.Tanh()
        self.fc1 = nn.Linear(hidden_size * 2,64)
        self.out = nn.Linear(64, num_classes)

    # lstm_output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
    # def attention_net(self, lstm_output, final_state):
    #     hidden = final_state.view(-1, self.hidden_size, 1)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
    #     attn_weights = torch.bmm(lstm_output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
    #     soft_attn_weights = F.softmax(attn_weights, 1)
    #     # [batch_size, n_hidden * num_directions(=2), n_step] * [batch_size, n_step, 1] = [batch_size, n_hidden * num_directions(=2), 1]
    #     context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
    #     return context, soft_attn_weights.data.numpy() # context : [batch_size, n_hidden * num_directions(=2)]

    def forward(self, X):
        # input = X.permute(1, 0, 2)
        # print('shape input',input.shape)
        # #input = X
        # # input : [len_seq, batch_size, embedding_dim]
        # hidden_state = Variable(torch.zeros(1, X.size(1), self.hidden_size)) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        # cell_state = Variable(torch.zeros(1, X.size(1), self.hidden_size)) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]

        # print('hidden shape',hidden_state.shape)

        # # out, (hidden, cell) = self.lstm(X)
        # # final_hidden_state, final_cell_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        # output, (final_hidden_state, final_cell_state) = self.lstm(input, (hidden_state, cell_state))

        # output, (final_hidden_state, cell) = self.lstm(X) 
        # print(output.shape)

        # output = output.permute(1, 0, 2) # output : [batch_size, len_seq, n_hidden]
        # attn_output, attention = self.attention_net(output, final_hidden_state)
        # return self.out(attn_output), attention # model : [batch_size, num_classes], attention : [batch_size, n_step]
        print(X.shape)
        H,_ = self.lstm(X)
        M = self.tanh1(H)
        print(H.shape,M.shape,self.w.shape)
        alpha = tf.softmax(torch.chain_matmul(M,self.w),dim=1).unsqueeze(-1)
        out = H * alpha
        out = torch.sum(out,1)
        out = F.relu(out)
        out = self.fc1(out)
        out = self.out(out)
        return out



def rnnatt(train_data: TrainData, n_targs: int, hidden_size:int, T:int, learning_rate=0.001, batch_size=128):
    # 定义配置器 T=>滑窗长度 截取前70%的数据作为训练集
    train_cfg = TrainConfig(
        T, int(train_data.feats.shape[0] * 0.7), batch_size, nn.MSELoss())
    print('train size: ',train_cfg.train_size)


    input_size = train_data.feats.shape[1] 
    print('input size: ',input_size +1)
    # 初始化网络结构
    # input_size: int, hidden_size: int, T: int,bidirectional: False,num_classes: int
    rnnatt_args = {
        "input_size" :input_size + 1,
        "hidden_size" : hidden_size,
        'bidirectional' :  False,
        "T" : 10,
        "num_classes" : 1,
    }
    print ("run args: ", rnnatt_args)
    rnn  = BiLSTM_Attention(**rnnatt_args)
    with open( ('../data/lstmatt.json'),"w") as fi: 
        json.dump(rnnatt_args,fi,indent=4)

    rnn_optimizer = optim.Adam(
        params=rnn.parameters(),
        lr=learning_rate
    )
    # 返回的网络结构
    rnnatt_net = RnnAttNet(
        rnn,rnn_optimizer
    )
    return train_cfg, rnnatt_net

In [49]:
# 初始化模型参数
init_args = {"batch_size": 128, "T": 10}

rnn_kwargs = rnn_att_kwargs = init_args

config, model = rnn(trainData, n_targs=len(targ_cols),hidden_size=64,T=10,learning_rate=.001,batch_size=128)

config1, model1 = rnnatt(trainData, n_targs=len(targ_cols),hidden_size=128,T=10,learning_rate=.001,batch_size=128)

train size:  700
input size:  6
run args:  {'input_size': 6, 'hidden_size': 64, 'num_layers': 1, 'output_size': 1, 'dropout': 0, 'batch_first': True}
train size:  700
input size:  6
run args:  {'input_size': 6, 'hidden_size': 128, 'bidirectional': False, 'T': 10, 'num_classes': 1}


In [25]:
# 平滑处理序列 开始训练
import torch
from torch.autograd import Variable
def PrepareData(batch_idx,t_cfg,train_data):
    feats = np.zeros((len(batch_idx), t_cfg.T - 1 , train_data.feats.shape[1]))
    y_history = np.zeros((len(batch_idx) ,t_cfg.T - 1 , train_data.targs.shape[1]))
    y_target = train_data.targs[batch_idx + t_cfg.T]
    # 获取采样的batch_id的下标和值
    # 获取特征和标签的相应下标值
    for b_i, b_idx in enumerate(batch_idx):
        b_slc = slice(b_idx, b_idx + t_cfg.T-1)
        #print('b_i',b_i,'b_slc',b_slc)
        feats[b_i, :, :] = train_data.feats[b_slc, :]
        #print(y_history[b_i : ].shape,train_data.targs[b_slc].shape)
        y_history[b_i :] = train_data.targs[b_slc]

    return feats, y_history, y_target


def rnn_train_iteration(t_net: RnnNet, loss_func: typing.Callable, X, y_history, y_target):
    input_data = np.append(X, y_history, axis=2)
    data1 = torch.from_numpy(input_data).to(torch.float32)
    #print('input shape: ',data1.shape)
    pred = t_net.rnn(Variable(data1))
   # print('pred shape: ',pred.shape)
    #pred = pred[0, :, :]
    label = torch.from_numpy(y_target).to(torch.float32).unsqueeze(1)
   # print('label shape: ',label.shape)
    loss = loss_func(pred, label)
    t_net.rnn_optimizer.zero_grad()
    loss.backward()
    t_net.rnn_optimizer.step()
    return loss.item()


In [35]:
'''
utils
'''

import logging
import os

import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable

import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def setup_log(tag='VOC_TOPICS'):
    # create logger
    logger = logging.getLogger(tag)
    # logger.handlers = []
    logger.propagate = False
    logger.setLevel(logging.DEBUG)
    # create console handler and set level to debug
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    # add formatter to ch
    ch.setFormatter(formatter)
    # add ch to logger
    # logger.handlers = []
    logger.addHandler(ch)
    return logger


def save_or_show_plot(file_nm: str, save: bool):
    if save:
        plt.savefig(file_nm)
    else:
        plt.show()


def numpy_to_tvar(x):
    return Variable(torch.from_numpy(x).type(torch.FloatTensor).to(device))


from sklearn.utils.validation import check_consistent_length, check_array

def mean_absolute_percentage_error(y_true, y_pred,
                                   sample_weight=None,
                                   multioutput='uniform_average'):
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
        y_true, y_pred, multioutput)
    check_consistent_length(y_true, y_pred, sample_weight)
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)
    output_errors = np.average(mape,
                               weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors
        elif multioutput == 'uniform_average':
            # pass None as weights to np.average: uniform mean
            multioutput = None

    return np.average(output_errors, weights=multioutput)

def _check_reg_targets(y_true, y_pred, multioutput, dtype="numeric"):
    check_consistent_length(y_true, y_pred)
    y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)

    if y_true.ndim == 1:
        y_true = y_true.reshape((-1, 1))

    if y_pred.ndim == 1:
        y_pred = y_pred.reshape((-1, 1))

    if y_true.shape[1] != y_pred.shape[1]:
        raise ValueError("y_true and y_pred have different number of output "
                         "({0}!={1})".format(y_true.shape[1], y_pred.shape[1]))

    n_outputs = y_true.shape[1]
    allowed_multioutput_str = ('raw_values', 'uniform_average',
                               'variance_weighted')
    if isinstance(multioutput, str):
        if multioutput not in allowed_multioutput_str:
            raise ValueError("Allowed 'multioutput' string values are {}. "
                             "You provided multioutput={!r}".format(
                                 allowed_multioutput_str,
                                 multioutput))
    elif multioutput is not None:
        multioutput = check_array(multioutput, ensure_2d=False)
        if n_outputs == 1:
            raise ValueError("Custom weights are useful only in "
                             "multi-output cases.")
        elif n_outputs != len(multioutput):
            raise ValueError(("There must be equally many custom weights "
                              "(%d) as outputs (%d).") %
                             (len(multioutput), n_outputs))
    y_type = 'continuous' if n_outputs == 1 else 'continuous-multioutput'

    return y_type, y_true, y_pred, multioutput

import matplotlib.pyplot as plt
logger = setup_log()
from sklearn.metrics import mean_squared_error,mean_absolute_error


def adjust_learning_rate(net: RnnNet, n_iter: int):
    # TODO: Where did this Learning Rate adjustment schedule come from?
    # Should be modified to use Cosine Annealing with warm restarts https://www.jeremyjordan.me/nn-learning-rate/
    if n_iter % 10000 == 0 and n_iter > 0:
        for enc_params, dec_params in zip(net.enc_opt.param_groups, net.dec_opt.param_groups):
            enc_params['lr'] = enc_params['lr'] * 0.9
            dec_params['lr'] = dec_params['lr'] * 0.9

'''
预测代码
'''
def predict(t_net: RnnNet, t_dat: TrainData, train_size: int, batch_size: int, T: int, on_train=False):
    # 设置输出的维度
    out_size = t_dat.targs.shape[1]

    if on_train:
        y_pred = np.zeros((train_size - T + 1, out_size))
    else:
        # 如果是测试就设置输出的长度为测试数据长
        y_pred = np.zeros((t_dat.feats.shape[0] - train_size, out_size))

    '''
    以每一次batch大小重新批量预测数据
    '''
    for y_i in range(0, len(y_pred), batch_size):
        y_slc = slice(y_i, y_i + batch_size)
        batch_idx = range(len(y_pred))[y_slc]
        X = np.zeros((len(batch_idx), T - 1, t_dat.feats.shape[1] ))
        y_history = np.zeros((len(batch_idx), T - 1, t_dat.targs.shape[1]))

        for b_i, b_idx in enumerate(batch_idx):
            if on_train:
                idx = range(b_idx, b_idx + T - 1)
            else:
                idx = range(b_idx + train_size - T, b_idx + train_size - 1)

            X[b_i, :, :] = t_dat.feats[idx, :]
            y_history[b_i, :] = t_dat.targs[idx]

        '''
        在这里重新合并rnn数据格式
        '''
        input_data = torch.from_numpy(np.append(X, y_history, axis=2)).to(torch.float32)
        y_pred[y_slc] = t_net.rnn(Variable(input_data)).detach().numpy()
    return y_pred





In [50]:
def train_rnn(net: RnnNet, train_data: TrainData, t_cfg: TrainConfig, n_epochs, save_plots=False):
    # 完成所有数据训练的迭代次数
    iter_per_epoch = int(np.ceil(t_cfg.train_size * 1. / t_cfg.batch_size))
    print('iter_per_epoch: ',t_cfg.train_size,t_cfg.batch_size,iter_per_epoch)
    # 存储损失值列表
    iter_losses = np.zeros(n_epochs * iter_per_epoch)
    print('iter_losses: ',iter_losses.shape)
    # 存储每次epoch的损失值
    epoch_losses = np.zeros(n_epochs)
   
    n_iter = 0

    for e_i in range(n_epochs):
        # 随机生成
        perm_idx = np.random.permutation(t_cfg.train_size - t_cfg.T)

        # 循环迭代 每次迭代的步长为batch_size，每次选择batch_size大小的数据进行预测
        for t_i in range(0, t_cfg.train_size, t_cfg.batch_size):
            # 随机选择索引列
            batch_idx = perm_idx[t_i:(t_i + t_cfg.batch_size)]
            # TODO 闭包函数，返回处理好的特征，历史数据，预测目标值
            feats,y_history,y_target = PrepareData(batch_idx,t_cfg,train_data)
            # data,label = ProcessData(feats,y_history,y_target)
            # 使用新的训练函数
            loss = rnn_train_iteration(net, t_cfg.loss_func,
                                       feats, y_history, y_target)
                                    
            iter_losses[e_i * iter_per_epoch + t_i // t_cfg.batch_size] = loss
            # if (j / t_cfg.batch_size) % 50 == 0:
            #    self.logger.info("Epoch %d, Batch %d: loss = %3.3f.", i, j / t_cfg.batch_size, loss)
            n_iter += 1
            adjust_learning_rate(net, n_iter)
            
        epoch_losses[e_i] = np.mean(
            iter_losses[range(e_i * iter_per_epoch, (e_i + 1) * iter_per_epoch)])
        print ('epoch loss: ',epoch_loss[e_i])
        if e_i % 10 == 0:
            with torch.no_grad():
                y_test_pred = predict(net, train_data,
                                    t_cfg.train_size, t_cfg.batch_size, t_cfg.T,
                                    on_train=False)
                # TODO: make this MSE and make it work for multiple inputs
                val_loss = y_test_pred - train_data.targs[t_cfg.train_size:]
                logger.info(
                    f"Epoch {e_i:d}, train loss: {epoch_losses[e_i]:3.3f}, val loss: {np.mean(np.abs(val_loss))}.")
                
                y_train_pred = predict(net, train_data,
                                    t_cfg.train_size, t_cfg.batch_size, t_cfg.T,
                                    on_train=True)
                plt.figure()
                plt.plot(range(1, 1 + len(train_data.targs)), train_data.targs,
                        label="True")
                plt.plot(range(t_cfg.T, len(y_train_pred) + t_cfg.T), y_train_pred,
                        label='Predicted - Train')
                plt.plot(range(t_cfg.T + len(y_train_pred), len(train_data.targs) + 1), y_test_pred,
                        label='Predicted - Test')
                plt.legend(loc='upper left')

                # print(1,1 + len(train_data.targs))
                # print(t_cfg.T,len(y_train_pred) + t_cfg.T)
                # print(t_cfg.T + len(y_train_pred),len(train_data.targs) +1)


                # todo 计算三者最后的MSE MAE MAPE
                y_test_list = list(y_test_pred)
                y_real = list(train_data.targs)[t_cfg.T + len(y_train_pred)-1 :len(train_data.targs) ]
                # print(len(y_real),len(y_test_list))


                print('rmse: ',np.sqrt(mean_squared_error(y_real,y_test_list)))
                print('mae: ', mean_absolute_error(y_real, y_test_list))
                print('mape: ', mean_absolute_percentage_error(y_real, y_test_list))
                save_or_show_plot(f"pred_rnn{e_i}.png", save_plots)

    print('epoch_loss',epoch_losses)
    print('训练结束')
    return iter_losses, epoch_losses

print('model params: ',model.rnn)
iter_loss, epoch_loss = train_rnn(
            model1, trainData, config1, n_epochs=30, save_plots=True)

model params:  Lstm(
  (rnn): LSTM(6, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=1, bias=True)
)
iter_per_epoch:  700 128 6
iter_losses:  (180,)
torch.Size([128, 9, 6])
torch.Size([128, 9, 128]) torch.Size([128, 9, 128]) torch.Size([256])


RuntimeError: Tensor dimension is 3, expected 2 instead.